<a href="https://colab.research.google.com/github/ranwiththecode/high-fantasy-data-analysis/blob/main/mining_current.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install required packages
!pip install requests pandas tqdm

import requests
import json
import time
import pandas as pd
from tqdm import tqdm
from google.colab import drive

BOOK_TITLE = "eragon_paolini"  # Replace with your book's title

# Mount Google Drive
drive.mount('/content/drive')

def get_all_reviews(api_url, initial_payload, headers):
    all_reviews = []
    page_count = 0

    while True:
        try:
            response = requests.post(api_url, headers=headers, json=initial_payload, timeout=15)
            data = response.json()

            # Skip if error in response
            if 'errors' in data:
                print(f"Skipping page due to error: {data['errors']}")
                break

            reviews = data.get('data', {}).get('getReviews', {}).get('edges', [])

            # Process reviews with error handling
            for review in reviews:
                try:
                    # Handle missing creator data
                    creator = review['node']['creator'] or {
                        'name': 'Anonymous',
                        'imageUrlSquare': None
                    }

                    all_reviews.append({
                        **review['node'],
                        'creator': creator
                    })
                except KeyError as e:
                    print(f"Skipping malformed review: {e}")
                    continue

            # Pagination logic
            page_info = data['data']['getReviews']['pageInfo']
            if not page_info.get('nextPageToken'):
                break

            initial_payload['variables']['pagination']['after'] = page_info['nextPageToken']
            page_count += 1
            time.sleep(1.5)

        except Exception as e:
            print(f"Error: {e}. Retrying...")
            time.sleep(5)
            continue

    return all_reviews

# Configuration
config = {
    "api_url": "https://kxbwmqov6jgg3daaamb744ycu4.appsync-api.us-east-1.amazonaws.com/graphql",
    "headers": {
        "Content-Type": "application/json",
        "X-Api-Key": "da2-xpgsdydkbregjhpr6ejzqdhuwy"
    },
    "payload_template": {
        "operationName": "getReviews",
        "query": """query getReviews($filters: BookReviewsFilterInput!, $pagination: PaginationInput) {
            getReviews(filters: $filters, pagination: $pagination) {
                totalCount
                edges {
                    node {
                        id
                        creator { name imageUrlSquare }
                        text
                        rating
                        createdAt
                        updatedAt
                    }
                }
                pageInfo { nextPageToken }
            }
        }""",
        "variables": {
            "filters": {
                "resourceType": "WORK",
                "resourceId": "kca://work/amzn1.gr.work.v1.xpxwQN_UlodQc9lPpilxhg"
            },
            "pagination": {"limit": 30}
        }
    }
}

# Run the scraper
print("🚀 Starting review collection...")
reviews = get_all_reviews(
    config["api_url"],
    config["payload_template"],
    config["headers"]
)

# Process and save results
if reviews:
    print(f"\n📊 Success! Collected {len(reviews)} reviews.")

    df = pd.json_normalize([{
        **r['node'],
        'creator_name': r['node']['creator']['name'],
        'creator_image': r['node']['creator']['imageUrlSquare']
    } for r in reviews])

    # Save to Drive
    # Replace your save code with this:
save_path = '/content/drive/MyDrive/Goodreads_Data/'
!mkdir -p "{save_path}"

# Clean the title (replace spaces with underscores)
clean_title = BOOK_TITLE.replace(" ", "_")

df.to_csv(f'{save_path}{clean_title}_reviews.csv', index=False)
df.to_json(f'{save_path}{clean_title}_reviews.json', indent=2)

print(f"Saved as:\n{save_path}{clean_title}_reviews.csv\n{save_path}{clean_title}_reviews.json")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
🚀 Starting review collection...


📖 Collected 600 reviews so far...


📖 Collected 1200 reviews so far...


📖 Collected 1800 reviews so far...


📖 Collected 2400 reviews so far...


...

📖 Collected 22798 reviews so far...


📖 Collected 23398 reviews so far...


📖 Collected 23998 reviews so far...


📖 Collected 24598 reviews so far...


⚠️ API Error: [{'path': ['getReviews', 'edges', 21, 'node', 'creator'], 'data': None, 'errorType': 'MappingTemplate', 'errorInfo': None, 'locations': [{'line': 7, 'column': 25, 'sourceName': None}], 'message': 'Template transformation yielded an empty response.'}]

📊 Success! Collected 24838 reviews.
Saved as:
/content/drive/MyDrive/Goodreads_Data/eragon_paolini_reviews.csv
/content/drive/MyDrive/Goodreads_Data/eragon_paolini_reviews.json


In [ ]:
# ===== 1. RESET ENVIRONMENT =====
# Clean slate installation
!pip install --upgrade --force-reinstall \
    numpy==2.0.2 \
    scipy==1.13.1 \
    pandas==2.2.2 \
    google-cloud-bigquery==3.31.0 \
    beautifulsoup4==4.12.3 \
    emoji==2.11.0 \
    textblob==0.17.1 \
    vaderSentiment==3.3.2 \
    gensim==4.3.2 \
    pyLDAvis==3.4.1 \
    --quiet

# Verify core packages
import numpy, scipy, pandas
print(f"Versions: numpy {numpy.__version__}, scipy {scipy.__version__}, pandas {pandas.__version__}")

# ===== 2. FIX SPACY CONFLICT =====
# Isolate our environment (optional but recommended)
!python -m pip install --user --ignore-installed virtualenv
!python -m virtualenv colab_env
!source colab_env/bin/activate

# ===== 3. DRIVE MOUNT =====
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
print("✅ Environment ready")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 49.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 39.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.1/250.1 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.8/160.8 kB 401.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.1/216.1 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 38.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2

In [ ]:
# ===== 1. CLEAN INSTALLATION =====
# First uninstall problematic packages
!pip uninstall -y numpy scipy pandas gensim

# Install stable versions
!pip install \
    numpy==1.26.4 \
    scipy==1.13.1 \
    pandas==2.2.2 \
    google-cloud-bigquery==3.31.0 \
    beautifulsoup4==4.12.3 \
    emoji==2.11.0 \
    textblob==0.17.1 \
    vaderSentiment==3.3.2 \
    gensim==4.3.2 \
    pyLDAvis==3.4.1 \
    --quiet

# ===== 2. VERIFY INSTALLS =====
import numpy as np
import scipy
import pandas as pd
from scipy.linalg import triu  # Previously problematic import

print(f"Versions: numpy {np.__version__}, scipy {scipy.__version__}, pandas {pd.__version__}")
print("✅ All packages installed correctly")

# ===== 3. MOUNT DRIVE =====
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Successfully uninstalled numpy-1.26.4
Found existing installation: scipy 1.13.1
Uninstalling scipy-1.13.1:
  Successfully uninstalled scipy-1.13.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spacy 3.8.7 requires thinc<8.4.0,>=8.3.4, which is not installed.
google-colab 1.0.0 requires google-auth==2.38.0, but you have google-auth 2.40.3 which is incompatible.
google-colab 1.0.0 requires requests==2.32.3, but you have requests 2.32.4 which is incompatible.
sklearn-compat 0.1.3 requires scikit-learn<1.7,>=1.2, but you have scikit-learn 1.7.0 which is incompatible.
tensorflow 2.18.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<6.0.0dev,>=3.20.3, but you have protobuf 6.31.1 which is incompatible.
ydf 0.12.0 requires protobuf<6.0.0,>=5.29.1, but you have protobuf 6.

ImportError: cannot import name 'triu' from 'scipy.linalg' (/usr/local/lib/python3.11/dist-packages/scipy/linalg/__init__.py)